<a href="https://colab.research.google.com/github/nahbos/AUT-Language-Understanding/blob/main/Ex02/bert_slot_intent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sobhan Moradian Daghigh
### **Ex 02: Slot Filling & Intent detection**

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.7 MB/s 
     |████████████████████████████████| 182 kB 60.2 MB/s 
     |████████████████████████████████| 7.6 MB 49.9 MB/s 


In [6]:
# Load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from transformers import BertTokenizer, TFBertModel
from urllib.request import urlretrieve

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam

In [1]:
!wget -nc https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Ex02/dataset/development-en.conllu
!wget -nc https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Ex02/dataset/test-en.conllu
!wget -nc https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Ex02/dataset/train-en.conllu

--2022-12-08 13:49:18--  https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Ex02/dataset/development-en.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1492856 (1.4M) [text/plain]
Saving to: ‘development-en.conllu’

development-en.conl 100%[===================>]   1.42M  --.-KB/s    in 0.02s   

2022-12-08 13:49:18 (69.5 MB/s) - ‘development-en.conllu’ saved [1492856/1492856]

--2022-12-08 13:49:18--  https://raw.githubusercontent.com/nahbos/AUT-Language-Understanding/main/Ex02/dataset/test-en.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP req

In [7]:
lines_train = Path('./train-en.conllu').read_text('utf-8').strip().splitlines()

In [10]:
print(lines_train[0])

# text: tell me the weather report for half moon bay


In [50]:
def data_preparing(dataset_path):
    prepared = []
    data_list = set(Path(dataset_path).read_text('utf-8').strip().split('\n\n'))
    for data in data_list:
        data_dic = ''
        for line in data.split('\n')[3:]:
            bio = line.split('\t')[-1]    # BIO
            bio = bio if bio != 'NoLabel' else 'O'
            tokens = line.split('\t')[1]  # tokens
            intent = data.split('\n')[1].split(':')[1].strip()  # intent
            data_dic += (tokens + ":" + bio + ' ')
        data_dic += '<=> ' + intent
        prepared.append(data_dic)
    return prepared

In [54]:
train = data_preparing('./train-en.conllu')
val   = data_preparing('./development-en.conllu')
test  = data_preparing('./test-en.conllu')

In [55]:
train[:10]

['cancel:O reminder:B-reminder/noun for:O school:B-reminder/todo meeting:I-reminder/todo <=> reminder/cancel_reminder',
 'set:O an:O alarm:O for:B-datetime 3:I-datetime pm:I-datetime every:I-datetime work:I-datetime day:I-datetime <=> alarm/set_alarm',
 'what:O is:O the:O weather:B-weather/noun in:O clearwater:B-location <=> weather/find',
 'set:O the:O alarm:O for:B-datetime 20:I-datetime minutes:I-datetime from:I-datetime now:I-datetime <=> alarm/set_alarm',
 'delete:O 8:B-datetime a.m:I-datetime alarm:O .:O <=> alarm/cancel_alarm',
 'what:O kind:O of:O weather:B-weather/noun am:O i:O going:O to:O have:O in:O san:B-location diego:I-location this:B-datetime weekend:I-datetime <=> weather/find',
 'is:O it:O going:O to:O be:O hot:B-weather/attribute this:B-datetime evening:I-datetime ?:O <=> weather/find',
 'set:O alarm:O every:B-datetime minute:I-datetime for:I-datetime 30:I-datetime minutes:I-datetime <=> alarm/set_alarm',
 'set:O two:O alarms:O .:O one:O for:B-datetime 8:I-datetime :

In [19]:
tt = Path('./train-en.conllu').read_text('utf-8').strip().split('\n\n')

In [24]:
tt[0].split('\n')

['# text: tell me the weather report for half moon bay',
 '# intent: weather/find',
 '# slots: 12:26:weather/noun,31:44:location',
 '1\ttell\tweather/find\tNoLabel',
 '2\tme\tweather/find\tNoLabel',
 '3\tthe\tweather/find\tNoLabel',
 '4\tweather\tweather/find\tB-weather/noun',
 '5\treport\tweather/find\tI-weather/noun',
 '6\tfor\tweather/find\tNoLabel',
 '7\thalf\tweather/find\tB-location',
 '8\tmoon\tweather/find\tI-location',
 '9\tbay\tweather/find\tI-location']

In [31]:
lis = []
for line in tt[0].split('\n')[3:]:
    bio = line.split('\t')[-1]
    lis.append(bio)

In [34]:
for line in tt[0].split('\n')[3:]:
    tokens = line.split('\t')[1] # tokens
    print(bio)

tell
me
the
weather
report
for
half
moon
bay


In [46]:
data_list = set(Path('./train-en.conllu').read_text('utf-8').strip().split('\n\n'))

In [48]:
data_list = Path('./train-en.conllu').read_text('utf-8').strip().split('\n\n')


In [49]:
len(data_list)

30521